In [0]:
%sql
CREATE TABLE departments (
    dept_id INT,
    dept_name STRING
);

INSERT INTO departments VALUES
(1, 'Engineering'),
(2, 'HR'),
(3, 'Sales'),
(4, 'Finance');

In [0]:
CREATE TABLE sales (
    sale_id INT,
    customer_id INT,
    amount DECIMAL(10,2),
    sale_date DATE
);
INSERT INTO sales VALUES
(101, 1, 250.00, '2026-01-05'),
(102, 1, 150.50, '2026-01-10'),
(103, 2, 500.00, '2026-01-07'),
(104, 2, 200.00, '2026-01-12'),
(105, 3, 300.00, '2026-01-08'),
(106, 4, 450.00, '2026-01-09'),
(107, 4, 100.00, '2026-01-15');

In [0]:
%sql
CREATE TABLE employees (
    emp_id INT,
    emp_name STRING,
    salary INT,
    dept_id INT,
    manager_id INT
);

INSERT INTO employees VALUES
(101, 'Alice', 120000, 1, NULL),   -- Head Engineering
(102, 'Bob', 80000, 1, 101),
(103, 'Charlie', 75000, 1, 101),
(104, 'David', 110000, 2, NULL),   -- Head HR
(105, 'Eva', 60000, 2, 104),
(106, 'Frank', 50000, 3, 108),     -- Reports to Hank
(107, 'Grace', 90000, NULL, 101),  -- No department
(108, 'Hank', 95000, 3, 101),
(109, 'Ivy', 75000, 3, 108),
(110, 'Jake', 75000, 3, 108),
(111, 'OrphanMgr', 70000, 4, 999); -- Invalid manager_id


In [0]:
%sql
CREATE TABLE employee_duplicates (
    emp_id INT,
    emp_name STRING
);

INSERT INTO employee_duplicates VALUES
(201, 'Tom'),
(202, 'Jerry'),
(201, 'Tom'),
(203, 'Anna'),
(202, 'Jerry');


In [0]:
%sql
CREATE TABLE customers (
    customer_id INT,
    customer_name STRING,
    city STRING
);

INSERT INTO customers VALUES
(1, 'John', 'New York'),
(2, 'Emma', 'Los Angeles'),
(3, 'Robert', 'Chicago'),
(4, 'Sophia', 'Houston'),
(5, 'Liam', 'Boston');  -- No orders


In [0]:
%sql
CREATE TABLE orders (
    order_id INT,
    customer_id INT,
    order_date DATE
);

INSERT INTO orders VALUES
(1001, 1, DATE('2024-01-01')),
(1002, 1, DATE('2024-01-05')),
(1003, 2, DATE('2024-02-10')),
(1004, 3, DATE('2024-02-15')),
(1005, 99, DATE('2024-03-01'));  -- Orphan customer


In [0]:
%sql
CREATE TABLE products (
    product_id INT,
    product_name STRING,
    category STRING
);

INSERT INTO products VALUES
(10, 'Laptop', 'Electronics'),
(11, 'Phone', 'Electronics'),
(12, 'Keyboard', 'Accessories'),
(13, 'Mouse', 'Accessories'),
(14, 'Tablet', 'Electronics');


In [0]:
%sql
CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    quantity INT,
    unit_price INT
);

INSERT INTO order_items VALUES
(1001, 10, 2, 1000),
(1001, 11, 1, 800),
(1002, 10, 1, 1000),
(1003, 12, 5, 200),
(1004, 13, 3, 300);


In [0]:
%sql
CREATE TABLE students (
    student_id INT,
    student_name STRING
);

INSERT INTO students VALUES
(1, 'Alex'),
(2, 'Brian'),
(3, 'Cathy');


In [0]:
%sql
CREATE TABLE courses (
    course_id INT,
    course_name STRING
);

INSERT INTO courses VALUES
(101, 'Math'),
(102, 'Physics'),
(103, 'Chemistry');


In [0]:
%sql
CREATE TABLE enrollments (
    student_id INT,
    course_id INT
);

INSERT INTO enrollments VALUES
(1, 101),
(1, 102),
(2, 101),
(3, 103);


In [0]:
%sql
CREATE TABLE employees_old (
    emp_id INT,
    emp_name STRING,
    salary INT
);

INSERT INTO employees_old VALUES
(101, 'Alice', 115000),   -- Salary changed
(102, 'Bob', 80000),
(103, 'Charlie', 75000),
(112, 'NewHire', 50000);  -- Missing in current table


In [0]:
%sql
SHOW TABLES

#Find employees who don’t have departments

In [0]:
select * from employees E
left join departments D
on D.dept_id=E.dept_id
where D.dept_id is null


# **JOINS**

#Find common records between two tables


In [0]:
select * from employees E
join departments D
on D.dept_id=E.dept_id


#Find unmatched records from both tables


In [0]:
SELECT 
    e.emp_id AS current_emp_id,
    o.emp_id AS old_emp_id,
    e.emp_name AS current_name,
    o.emp_name AS old_name
FROM employees e
FULL OUTER JOIN employees_old o
    ON e.emp_id = o.emp_id
WHERE e.emp_id IS NULL 
   OR o.emp_id IS NULL;



#Self join: employees and their managers


In [0]:
select * from employees e
join employees m
on e.manager_id=m.emp_id

#Find duplicate records using joins

In [0]:
select * from employee_duplicates

In [0]:
with cte as(
select *,
row_number() over(order by emp_id desc) as rn
from employee_duplicates
)
select c1.emp_id,c2.emp_name
from cte c1
join cte c2
on c1.emp_id=c2.emp_id
and c1.emp_name=c2.emp_name
and c1.rn < c2.rn;

#Find customers who never placed an order

In [0]:
select * from customers

In [0]:
select * from orders

In [0]:
select * from customers c
left join orders o
on c.customer_id=o.customer_id
WHERE o.customer_id IS NULL;--if you are using left join,so all the values from the left table will always be there, therefore find the null values from the right table on the column which we used for joining

#Find top 3 highest salary employee from each department

In [0]:
SELECT *
FROM (
    SELECT 
        e.emp_id,
        e.emp_name,
        e.salary,
        d.dept_name,
        ROW_NUMBER() OVER (
            PARTITION BY e.dept_id
            ORDER BY e.salary DESC
        ) AS rn
    FROM employees e
    JOIN departments d
        ON e.dept_id = d.dept_id
) t
WHERE rn <= 3;
--When emp and dept are two diff tables. needed when info abput dept is required

In [0]:
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY dept_id
               ORDER BY salary DESC
           ) AS rn
    FROM employees
) t
WHERE rn <= 3;


In [0]:
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY dept_id
               ORDER BY salary DESC
           ) AS rn
    FROM employees
) t
WHERE rn <= 3;


#Find second highest salary using joins

In [0]:
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (ORDER BY salary DESC) AS rn
    FROM employees
) t
WHERE rn = 3;


#Find employees earning more than their manager

In [0]:
SELECT e.emp_id AS emp_id,
       e.emp_name AS employee_name,
       e.salary AS emp_salary,
       m.emp_id AS mgr_id,
       m.emp_name AS manager_name,
       m.salary AS mgr_salary
FROM employees e
JOIN employees m 
    ON e.manager_id = m.emp_id
WHERE e.salary > m.salary;


#Calculate total sales per customer

In [0]:
select * from sales

In [0]:
select * from customers

In [0]:
SELECT 
    c.customer_id,
    c.customer_name,
    SUM(s.amount) AS total_sales
FROM sales s
JOIN customers c
    ON c.customer_id = s.customer_id
GROUP BY c.customer_id, c.customer_name;
--All the columns in the group by should be in the select statement

#Find missing records between two datasets

In [0]:
select emp_id,emp_name from employees where emp_id not in(select emp_id from Employees_old)
Union 
select emp_id,emp_name from Employees_old where emp_id not in(select emp_id from employees)

In [0]:
SELECT e.emp_id,e.emp_name
FROM employees e
LEFT JOIN Employees_old o
    ON e.emp_id = o.emp_id
WHERE o.emp_id IS NULL

UNION

SELECT o.emp_id,o.emp_name
FROM Employees_old o
LEFT JOIN employees e
    ON o.emp_id = e.emp_id
WHERE e.emp_id IS NULL;

In [0]:
select * from employees

In [0]:
select * from employees_old

#Identify orphan records using joins

In [0]:
SELECT *
FROM employees
WHERE manager_id IS NOT NULL
AND manager_id NOT IN (
    SELECT emp_id FROM employees
);

#Compare two tables and show differences

In [0]:
SELECT emp_id,emp_name FROM employees
EXCEPT
SELECT emp_id,emp_name FROM employees_old

UNION ALL

SELECT emp_id,emp_name FROM employees_old
EXCEPT
SELECT emp_id,emp_name FROM employees;

#Find customers with no orders

In [0]:
SELECT *
FROM Customers c
LEFT JOIN Orders o
    ON c.customer_id = o.customer_id
WHERE o.customer_id IS NULL;

In [0]:
select * from customers

In [0]:
select * from orders

#Find duplicate records

In [0]:
select * from employee_duplicates

In [0]:
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY emp_id
               ORDER BY emp_id
           ) AS rn
    FROM employee_duplicate
) t
WHERE rn > 1;

#Second highest salary

In [0]:
SELECT *
FROM (
    SELECT *,
           DENSE_RANK() OVER (ORDER BY salary DESC) AS rnk
    FROM employees
) t
WHERE rnk = 2;

# **Window Function**